# Platfrom Testing

# Imports

In [1]:
import subprocess
import os.path
import time
import yaml

# Prepare paths and load yaml config file

In [2]:
# Prepare the paths that will be used during the test
yamlInfoFile="VMsSetup.yaml"
fileExt = ""
commandsFolder = ""
startCommand = ""
ProcessTail = ""
LogCommandTail = ""

with open(yamlInfoFile) as file:
    doc = yaml.load(file, Loader=yaml.FullLoader)

    for item, doc in doc.items():
        if item == "Setup":
            Setup = doc
        elif item == "Examples":
            Examples = doc    
        else:
            Machines = doc

if Setup['TestMgrOS'] == "Linux":
    fileExt = ".sh"
    commandsFolder = "Commands/Linux/"
    startCommand = "setsid"
    ScriptServerPath = "Scripts/Server/"
    ScriptDevicePath = "Scripts/Device/"
    DeviceFolderPath = "DeviceFolder/"
    dockerRegistryCertsPath = "DeviceFolder/dockerRegistryCerts/"
    ProcessTitle = ""
    LogsPath = "Logs/"
    SleepTime3Sec = "sleep 3\n"
    ExamplesPath = "examples/"
    AddScriptsServerPath = "Scripts/Server/"
    AddScriptsDevicePath = "Scripts/Device/"
    for VM in Machines:
        VM['Password'] = "\'{}\'".format(VM['Password'])
elif Setup['TestMgrOS'] == "Win":
    fileExt = ".bat"
    commandsFolder = "Commands\\Win\\"
    startCommand = "start"
    ScriptServerPath = "Scripts\\Server\\*"
    ScriptDevicePath = "Scripts\\Device\\*"
    DeviceFolderPath = "DeviceFolder\*"
    dockerRegistryCertsPath = "DeviceFolder\dockerRegistryCerts\*"
    ProcessTitle = "\"Running\""
    LogsPath = "Logs\\"
    SleepTime3Sec = "TIMEOUT 3 > nul\n"
    ExamplesPath = "examples\\*"
    AddScriptsServerPath = "Scripts\\Server\\"
    AddScriptsDevicePath = "Scripts\\Device\\"
    
batMoveFiles = "{}MoveFiles{}".format(commandsFolder, fileExt)
batDownloadExamples = "{}DownloadExamples{}".format(commandsFolder, fileExt)
batCompileExamples = "{}CompileExamples{}".format(commandsFolder, fileExt)
batIIPPrerequisiteInstallation = "{}IIPPrerequisiteInstallation{}".format(commandsFolder, fileExt)
batDockerInstallation = "{}DockerInstallation{}".format(commandsFolder, fileExt)
batInstallPlatform = "{}InstallPlatform{}".format(commandsFolder, fileExt)
#batDockerRegistryCerts = "{}DockerRegistryCerts{}".format(commandsFolder, fileExt)
batRunDockerRegistry = "{}RunDockerRegistry{}".format(commandsFolder, fileExt)
batPullRunEcsContainer = "{}PullRunEcsContainer{}".format(commandsFolder, fileExt)
batStopEcsContainer = "{}StopEcsContainer{}".format(commandsFolder, fileExt)
batStartPlatform = "{}StartPlatform{}".format(commandsFolder, fileExt)
batJoinPlatform = "{}JoinPlatform{}".format(commandsFolder, fileExt)
batCliContainersCommand = "{}CliContainersCommand{}".format(commandsFolder, fileExt)
batCliServicesCommand = "{}CliServicesCommand{}".format(commandsFolder, fileExt)
#batDistrDockerRegistryCerts = "{}DistrDockerRegistryCerts{}".format(commandsFolder, fileExt)
batLeavePlatform = "{}LeavePlatform{}".format(commandsFolder, fileExt)
batStopPlatform = "{}StopPlatform{}".format(commandsFolder, fileExt)
batRebootVMs = "{}RebootVMs{}".format(commandsFolder, fileExt)
batGetCompleteAlert = "{}GetCompleteAlert{}".format(commandsFolder, fileExt)
executionCompletedFile = "CompletedAlert.info"

print(Setup['TestMgrOS'])

Linux


# Prepare Dynamic Scripts 

In [3]:
# Prepare variables and clean or create empty the bat files
RegistryList = []
VMHomePath = ""
VMHomeFilesPath = ""
VMHomeExamplesPath = ""
VMFileExt = ""
moveFilesLine = ""
downloadExamplesLine = ""
compileExamplesLine = ""
IIPPrerequisiteInstallationLine = ""
installPlatformLine = ""
#dockerRegistryCertsLine = ""
runDockerRegistryLine = ""
pullRunEcsContainerLine = ""
stopEcsContainerLine = ""
#DistrDockerRegistryCertsLine = ""
startPlatformLine = ""
joinPlatformLine = ""
cliContainersCommandLine = ""
LeavePlatformLine = ""
stopPlatformLine = ""
rebootVMLine = ""

writeFile = open(batMoveFiles, "w")
writeFile.write(moveFilesLine)
writeFile.close()
writeFile = open(batDownloadExamples, "w")
writeFile.write(downloadExamplesLine)
writeFile.close()
writeFile = open(batCompileExamples, "w")
writeFile.write(compileExamplesLine)
writeFile.close()
writeFile = open(batIIPPrerequisiteInstallation, "w")
writeFile.write(IIPPrerequisiteInstallationLine)
writeFile.close()
writeFile = open(batInstallPlatform, "w")
writeFile.write(installPlatformLine)
writeFile.close()
#writeFile = open(batDockerRegistryCerts, "w")
#writeFile.write(dockerRegistryCertsLine)
#writeFile.close()
writeFile = open(batRunDockerRegistry, "w")
writeFile.write(runDockerRegistryLine)
writeFile.close()
writeFile = open(batPullRunEcsContainer, "w")
writeFile.write(pullRunEcsContainerLine)
writeFile.close()
writeFile = open(batStopEcsContainer, "w")
writeFile.write(stopEcsContainerLine)
writeFile.close()
#writeFile = open(batDistrDockerRegistryCerts, "w")
#writeFile.write(DistrDockerRegistryCertsLine)
#writeFile.close()
writeFile = open(batJoinPlatform, "w")
writeFile.write(joinPlatformLine)
writeFile.close()
writeFile = open(batCliContainersCommand, "w")
writeFile.write(cliContainersCommandLine)
writeFile.close()
writeFile = open(batStartPlatform, "w")
writeFile.write(startPlatformLine)
writeFile.close()
writeFile = open(batLeavePlatform, "w")
writeFile.write(LeavePlatformLine)
writeFile.close()
writeFile = open(batStopPlatform, "w")
writeFile.write(stopPlatformLine)
writeFile.close()
writeFile = open(batRebootVMs, "w")
writeFile.write(rebootVMLine)
writeFile.close()

for VM in Machines:
    
    # Skip the VMs with "Skip : True" in VMsSetup.yaml
    if not VM['Included']:
        continue
    if VM['OS'] == "Linux":
        VMHomeFilesPath = "Files/"
        VMFileExt = ".sh"
    elif VM['OS'] == "Win":
        VMHomeFilesPath = "Files\\\\"
        VMFileExt = ".bat"
          
    rebootVMLine="plink -t -batch -pw {} {}@{} {}RestartVM{} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"))
    moveFilesMkdirLine="plink -t -batch -pw {} {}@{} \"mkdir {}Files\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath)
    mkExecuteFilesLine="plink -t -batch -pw {} {}@{} \"chmod 755 {}Files/*{}\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath, VMFileExt)
    # Create the bat files for the Server
    if VM['Role'] == "Server":
        # Create bat file to move the auto scripts to the Server
        moveFilesLine="echo y | pscp -r -pw {} \"{}\" {}@{}:{}Files\n".format(VM['Password'], ScriptServerPath, VM['Username'], VM['IP'], VMHomePath)
        # Create bat file to download examples on server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}DownloadExamples_{}.log &".format(LogsPath, VM['IP'])
        downloadExamplesLine="{} {} plink -t -batch -pw {} {}@{} {}DownloadExamples{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
        # Create bat file for install the platform as Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Install_{}.log &".format(LogsPath, VM['IP'])
        if VM['RunAsContainer']:
            installPlatformLine=""
        else:
            installPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}IIPInstallationServer{} {} {} {} {} {} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['EcsSvcMgrContainers'], Setup['DockerRegistry'], Setup['IIPEasyTracing'], VM['Username'], Setup['ArtifactsFolder'], Setup['NewInstallFiles'], ProcessTail)
        # Create bat file to create docker registry certs
        #if VM['DockerRegistry']:
        #    dockerRegistryCertsLine="plink -t -batch -pw {} {}@{} {}DockerRegistryCerts{} {} \n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"))
        #    batfile = open(batDockerRegistryCerts, "a")
        #    batfile.write(dockerRegistryCertsLine)
        #    batfile.close()
        
        # Create bat file to start the platform Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Start_{}.log &".format(LogsPath, VM['IP'])
        startPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}StartAllPlatformWithBroker{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
        batfile = open(batStartPlatform, "a")
        batfile.write(startPlatformLine)
        batfile.close()
        batfile = open(batDownloadExamples, "a")
        batfile.write(downloadExamplesLine)
        batfile.close()
        # Create bat file to stop the platform Server
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Stop_{}.log &".format(LogsPath, VM['IP'])
        stopPlatformLine="plink -t -batch -pw {} {}@{} {}StopAllPlatformWithBroker{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
        batfile = open(batStopPlatform, "a")
        batfile.write(stopPlatformLine)
        batfile.close()
    # Create the bat files for the Devices
    elif VM['Role'] == "Device":
        # Create bat file to move the auto scripts to the Device(s)
        moveFilesLine="echo y | pscp -r -pw {} \"{}\" {}@{}:{}Files\n".format(VM['Password'], ScriptDevicePath, VM['Username'], VM['IP'], VMHomePath)
        # Create bat file for install the platform as Device
        if Setup['TestMgrOS'] == "Linux":
            ProcessTail = " > {}Install_{}.log &".format(LogsPath, VM['IP'])
        if VM['RunAsContainer']:
            installPlatformLine=""
            # Create bat file to pull and run Ecs container to join the platform
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Join_{}.log &".format(LogsPath, VM['IP'])
            pullRunEcsContainerLine="{} {} plink -t -batch -pw {} {}@{} {}PullRunEcsContainer{} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'], ProcessTail)
            writeFile = open(batJoinPlatform, "a")
            writeFile.write(pullRunEcsContainerLine)
            writeFile.close()
            # Create bat file to stop Ecs container on the Device(s)
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Leave_{}.log &".format(LogsPath, VM['IP'])
            stopEcsContainerLine="plink -t -batch -pw {} {}@{} {}StopEcsContainer{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
            writeFile = open(batLeavePlatform, "a")
            writeFile.write(stopEcsContainerLine)
            writeFile.close()
        else:
            # Create bat file to start and join ecs file to the platform
            installPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}IIPInstallationDevice{} {} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['NewInstallFiles'], VM['Username'], ProcessTail)
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Join_{}.log &".format(LogsPath, VM['IP'])
            joinPlatformLine="{} {} plink -t -batch -pw {} {}@{} {}JoinPlatform{} {} {} {} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), VM['DeviceID'], VM['NewDeviceFiles'], Setup['MgtUiServerIP'], ProcessTail)
            batfile = open(batJoinPlatform, "a")
            batfile.write(joinPlatformLine)
            batfile.close()
            # Create bat file to stop and leave device(s) from the platform
            if Setup['TestMgrOS'] == "Linux":
                ProcessTail = " > {}Leave_{}.log &".format(LogsPath, VM['IP'])
            LeavePlatformLine="plink -t -batch -pw {} {}@{} {}LeavePlatform{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
            batfile = open(batLeavePlatform, "a")
            batfile.write(LeavePlatformLine)
            batfile.close()
        # Create bat file to distribute container certs to the Device(s)
        #DistrDockerRegistryCertsLine="echo y | pscp -r -pw {} \"{}\" {}@{}:{}Files\n".format(VM['Password'], dockerRegistryCertsPath, VM['Username'], VM['IP'], VMHomePath)
        #writeFile = open(batDistrDockerRegistryCerts, "a")
        #writeFile.write(DistrDockerRegistryCertsLine)
        #writeFile.close()
    print(VM['Name'])
    print(VM['IP'])
    print("RunAsContainer: {}".format(VM['RunAsContainer']))
    batfile = open(batMoveFiles, "a")
    batfile.write(moveFilesMkdirLine)
    batfile.write(moveFilesLine)
    if VM['OS'] == "Linux":
        batfile.write(mkExecuteFilesLine)
    batfile.close()
    # Create bat file for Prerequisite Installation for the platform
    if Setup['TestMgrOS'] == "Linux":
        ProcessTail = " > {}Prerequisite_{}.log &".format(LogsPath, VM['IP'])
    if VM['RunAsContainer']:
        IIPPrerequisiteInstallationLine="{} {} plink -t -batch -pw {} {}@{} {}DockerInstallation{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
    else:
        IIPPrerequisiteInstallationLine="{} {} plink -t -batch -pw {} {}@{} {}IIPPrerequisiteInstallation{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), ProcessTail)
    batfile = open(batIIPPrerequisiteInstallation, "a")
    batfile.write(IIPPrerequisiteInstallationLine)
    batfile.close()
    # Create bat file to run docker registry
    if Setup['RunDockerRegistry']:
        if VM['DockerRegistry']:
            runDockerRegistryLine="plink -t -batch -pw {} {}@{} {}RunDockerRegistry{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'], Setup['DockerRegistryPort'])
            batfile = open(batRunDockerRegistry, "a")
            batfile.write(runDockerRegistryLine)
            batfile.close()
            RegistryList.append(VM['IP'])
        else:
            runDockerRegistryLine="plink -t -batch -pw {} {}@{} {}GetDockerRegistry{} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), Setup['DockerRegistry'])
            batfile = open(batRunDockerRegistry, "a")
            batfile.write(runDockerRegistryLine)
            batfile.close()
    batfile = open(batInstallPlatform, "a")
    batfile.write(installPlatformLine)
    batfile.close()
    writeFile = open(batRebootVMs, "a")
    writeFile.write(rebootVMLine)
    writeFile.close()

if Setup['TestMgrOS'] == "Linux":
    subprocess.run(['chmod', '-R', '755', 'Commands/Linux'])


Server 147.172.178.145 Platform
147.172.178.145
RunAsContainer: False
Server 147.172.178.147 Device02
147.172.178.147
RunAsContainer: False
Server 147.172.178.148 Device03
147.172.178.148
RunAsContainer: True


# Prepare Tests

In [4]:
# Prepare the paths that will be used during the test
yamlInfoFile="VMsSetup.yaml"
fileExt = ""
testsFolder = ""
testProcessTail = ""
testResultsFile = ""
batTestResources = ""
testResourcesLine = ""

if Setup['TestMgrOS'] == "Linux":
    fileExt = ".sh"
    testsFolder = "Tests/Linux/Commands/"
    resultsPath = "Tests/Linux/Results/"
elif Setup['TestMgrOS'] == "Win":
    fileExt = ".bat"
    testsFolder = "Tests\\Win\\Commands\\"
    resultsPath = "Tests\\Win\\Results\\"

for VM in Machines:
    
    # Skip the VMs with "Skip : True" in VMsSetup.yaml
    if not VM['Included']:
        continue
    if VM['Role'] == "Server":
        batTestResources = "{}TestResources{}".format(testsFolder, fileExt)
        testProcessTail = " > {}Test_{}.log &".format(resultsPath, "TestResources")
        testResultsFile = "{}Test_{}.log".format(resultsPath, "TestResources")
        testResourcesLine="plink -t -batch -pw {} {}@{} \"cd Files/Install/gen && ./cli.sh resources list\" {} \n".format(VM['Password'], VM['Username'], VM['IP'], testProcessTail)
        writeFile = open(batTestResources, "w")
        writeFile.write(testResourcesLine)
        writeFile.close()

print(Setup['TestMgrOS'])

Linux


# Prepare Examples Dynamic Scripts

In [5]:
# create script to download the examples
if Setup['DownloadExamples']:
    LinuxExamplesLine = []
    PlatformURLZip="curl https://codeload.github.com/iip-ecosphere/platform/zip/refs/heads/main -o Platform.zip\n"
    LinuxExamplesLine.append("cd Files")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append(PlatformURLZip)
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("unzip Platform.zip\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("echo $1 | sudo -S chown -R $USER examples\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("mkdir -p examples\n")
    LinuxExamplesLine.append("\n")
    
    for Ex in Examples:
        if not Ex['Included']:
            continue
        if not Ex['Redownload']:
            continue
        LinuxExamplesLine.append("rm -r examples/{}; mkdir -p examples/{}\n".format(Ex['Name'], Ex['Name']))
        LinuxExamplesLine.append("cp -r platform-main/platform/examples/{}/* examples/{}/\n".format(Ex['Name'], Ex['Name']))
        LinuxExamplesLine.append("\n")
        
    LinuxExamplesLine.append("rm Platform.zip\n")
    LinuxExamplesLine.append("\n")
    LinuxExamplesLine.append("rm -r platform-main\n")
    LinuxExamplesLine.append("\n")
    
    batLinuxExamples = "{}DownloadExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxExamples, "w")
    for Line in LinuxExamplesLine:
        batfile.write(Line)
    batfile.close()
    
    WinExamplesLine = []
    WinExamplesLine.append("cd Files")
    WinExamplesLine.append("\n")
    WinExamplesLine.append(PlatformURLZip)
    WinExamplesLine.append("\n")
    WinExamplesLine.append("tar xzpvf Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("if not exist examples mkdir examples\n")
    WinExamplesLine.append("\n")

    for Ex in Examples:
        if not Ex['Included']:
            continue
        if not Ex['Redownload']:
            continue
            
        WinExamplesLine.append("rmdir /s examples\{} /q\n".format(Ex['Name']))
        WinExamplesLine.append("mkdir examples\{}\n".format(Ex['Name']))
        WinExamplesLine.append("Xcopy platform-main\platform\examples\{} examples\{} /E /H /C /I\n".format(Ex['Name'], Ex['Name']))
        WinExamplesLine.append("\n")


    WinExamplesLine.append("del /f Platform.zip\n")
    WinExamplesLine.append("\n")
    WinExamplesLine.append("rmdir /s platform-main /q\n")
    WinExamplesLine.append("\n")

    batWinExamples = "{}DownloadExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinExamples, "w")
    for Line in WinExamplesLine:
        batfile.write(Line)
    batfile.close()
    
    for Ex in Examples:
        if not Ex['Included']:
            continue
        print(Ex['Name'])
else:
    batLinuxExamples = "{}DownloadExamples.sh".format(AddScriptsServerPath)
    batfile = open(batLinuxExamples, "w")
    batfile.write("")
    batfile.close()
    
    batWinExamples = "{}DownloadExamples.bat".format(AddScriptsServerPath)
    batfile = open(batWinExamples, "w")
    batfile.write("")
    batfile.close()

SimpleMesh
examples.KODEX
examples.python
examples.rtsa
examples.vdw


# Move The Files

In [8]:
# Run script to move the auto scripts to the VMs 
if Setup['MoveScripts']:
    for VM in Machines:
        if not VM['Included']:
            continue
        print(VM['Name'])
        status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        while status != 0:
            print(VM['IP'] + " is DOWN !, waiting...")
            status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        print(VM['IP'] + " is UP !")
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batMoveFiles], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batMoveFiles], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()

Server 147.172.178.145 Platform
147.172.178.145 is UP !
Server 147.172.178.147 Device02
147.172.178.147 is UP !
Server 147.172.178.148 Device03
147.172.178.148 is UP !


# Install The Prerequisite for The Platform

In [12]:
# Run script to install the platform on VMs as Server and Device(s)
if Setup['InstallPrerequisite']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batIIPPrerequisiteInstallation], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the prerequisite installation on VMs")
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batIIPPrerequisiteInstallation], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the prerequisite installation on VMs")
        ls_output.communicate()

    print("The prerequisite installation is completed on all VMs")

# Run Docker Registry

In [13]:
# Run script to create docker registry certs
if Setup['RunDockerRegistry']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batRunDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batRunDockerRegistry], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()

# Install The Platform

In [9]:
# Run script to install the platform on VMs as Server and Device(s)
if Setup['InstallPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batInstallPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the installation on VMs")
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batInstallPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("waiting to complete the installation on VMs")
        ls_output.communicate()

    print("The installation is completed on all VMs")

waiting to complete the installation on VMs
The installation is completed on all VMs


# Restart the VMs after the installation

In [ ]:
# Run script to restart VMs
if Setup['InstallPlatform']:
    #if Setup['TestMgrOS'] == "Linux":
        #ls_output=subprocess.Popen(['sh', batRebootVMs], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #ls_output.communicate()
    #elif Setup['TestMgrOS'] == "Win":
        #ls_output=subprocess.Popen([batRebootVMs], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #ls_output.communicate()
        
    time.sleep(10)
    for VM in Machines:
        if not VM['Included']:
            continue
        print(VM['Name'])
        status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        while status != 0:
            print(VM['IP'] + " is DOWN !, waiting...")
            status,result = subprocess.getstatusoutput("ping -c1 -w2 " + VM['IP'])
        print(VM['IP'] + " is UP !")

# Create Docker Registry Certs

In [ ]:
# Run script to create docker registry certs
#if Setup['DockerRegistryCerts']:
#    if Setup['TestMgrOS'] == "Linux":
#        ls_output=subprocess.Popen(['sh', batDockerRegistryCerts], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#        ls_output.communicate()
#    elif Setup['TestMgrOS'] == "Win":
#        ls_output=subprocess.Popen([batDockerRegistryCerts], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#        ls_output.communicate()

# Get and Compile the examples

In [7]:
# Run script to compile the examples
if Setup['DownloadExamples']:
    # Download the examples
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batDownloadExamples], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batDownloadExamples], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()

if Setup['CompileExamples']:
    # Create and run script to compile the examples on the Server
    compileExamplesLine = ""
    writeFile = open(batCompileExamples, "w")
    writeFile.write(compileExamplesLine)
    writeFile.close()

    for Ex in Examples:
        if not Ex['Included']:
            continue

        exampleName = Ex['Name']
        
        if Ex['Compile'] == 'Normal':
            for VM in Machines:
                if not VM['Included']:
                    continue
                if VM['Role'] == "Server":
                    if VM['OS'] == "Linux":
                        VMHomeFilesPath = "Files/"
                        VMFileExt = ".sh"
                        VMHomeExamplesPath = "Files/examples/"
                        if Ex['Name'] == "examples.KODEX":
                            exampleName = Ex['Name'] + "/" + Ex['Name']
                    elif VM['OS'] == "Win":
                        VMHomeFilesPath = "Files\\\\"
                        VMFileExt = ".bat"
                        VMHomeExamplesPath = "Files\\\\examples\\\\"                        
                        if Ex['Name'] == "examples.KODEX":
                            exampleName = Ex['Name'] + "\\\\" + Ex['Name']
                    
                    if Setup['TestMgrOS'] == "Linux":
                        ProcessTail = " > {}Compile_{}_{}.log &".format(LogsPath, Ex['Name'], VM['IP'])
                    compileExamplesLine="{} {} plink -t -batch -pw {} {}@{} {}CompileExamplesNormal{} {} {}{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), VMHomeExamplesPath, exampleName, Setup['ArtifactsFolder'], ProcessTail)
                    writeFile = open(batCompileExamples, "a")
                    writeFile.write(compileExamplesLine)
                    writeFile.close()
            
        elif Ex['Compile'] == 'Connector':
            for VM in Machines:
                if not VM['Included']:
                    continue
                if VM['Role'] == "Server":
                    if VM['OS'] == "Linux":
                        VMHomeFilesPath = "Files/"
                        VMFileExt = ".sh"
                        VMHomeExamplesPath = "Files/examples/"
                    elif VM['OS'] == "Win":
                        VMHomeFilesPath = "Files\\\\"
                        VMFileExt = ".bat"
                        VMHomeExamplesPath = "Files\\\\examples\\\\"  
                        
                    if Setup['TestMgrOS'] == "Linux":
                        ProcessTail = " > {}Compile_{}_{}.log &".format(LogsPath, Ex['Name'], VM['IP'])
                    compileExamplesLine="{} {} plink -t -batch -pw {} {}@{} {}CompileExamplesConnector{} {} {}{} {} {}\n".format(startCommand, ProcessTitle, VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VM['Password'].replace("$","\\$"), VMHomeExamplesPath, exampleName, Setup['ArtifactsFolder'], ProcessTail)
                    writeFile = open(batCompileExamples, "a")
                    writeFile.write(compileExamplesLine)
                    writeFile.close()
                    
        elif Ex['Compile'] == 'No':
            continue
    
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batCompileExamples], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batCompileExamples], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()


# Start The Platform

In [17]:
# Run script to start the platform Server
if Setup['StartPlatfrom']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batStartPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batStartPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #ls_output.communicate()

    time.sleep(3)
    
    for VM in Machines:
        if VM['Role'] == "Server":
            if not VM['Included']:
                continue
            PlatformReady = 0
            while PlatformReady == 0:
                with open("{}Start_{}.log".format(LogsPath, VM['IP'])) as f:
                    if 'Broker and Platform are Running... Please don\'t close it' in f.read():
                        PlatformReady = 1
                    else:
                        print("Waiting for the platform on {} to start".format(VM['IP']))
                        time.sleep(3)
            print("The platform on {} is started successfully".format(VM['IP']))

Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
Waiting for the platform on 147.172.178.145 to start
The platform on 147.172.178.145 is started successfully


# Device(s) Join The Platform

In [18]:
# Run script to join the Device(s) to the platform
if Setup['DevicesJoinPlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batJoinPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batJoinPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(3)
    
    for VM in Machines:
        VM['ResourceID'] = ""
        if VM['Role'] == "Device":
            if not VM['Included']:
                continue

            DeviceJoin = 0
            DeviceID = ""
            while DeviceJoin == 0:
                with open("{}Join_{}.log".format(LogsPath, VM['IP'])) as f:
                    content = f.read()
                    if 'Ecs is Running... Please don\'t close it' in content:
                        DeviceJoin = 1
                    elif 'Ecs and ServiceMgr are Running... Please don\'t close it' in content:
                        DeviceJoin = 1
                    else:
                        print("Waiting for the device {} to join the platform".format(VM['IP']))
                        time.sleep(3)
            with open("{}Join_{}.log".format(LogsPath, VM['IP']), 'r')  as f:
                for line in f:
                    pass
                DeviceID = line
            DeviceIDInfo = DeviceID.split(' ')
            VM['ResourceID'] = DeviceIDInfo[1].replace('\r', '').replace('\n', '')
            print("The device {} is joined the platform successfully".format(VM['IP']))
       

Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
Waiting for the device 147.172.178.147 to join the platform
The device 147.172.178.147 is joined the platform successfully
The device 147.172.178.148 is joined the platform successfully


# Start Service Manager Container

In [12]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['StartServiceMgrContainer']:
    cliContainersCommandLine = "" 
    writeFile = open(batCliContainersCommand, "w")
    writeFile.write(cliContainersCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        ServiceMgrDesc = "http://{}:4200/download/SimpleMeshTestingApp_dflt_mesh-info.yml".format(Setup['MgtUiServerIP'])
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    if VMDevice['RunAsContainer']:
                        ProcessTail = ""
                        cliContainersCommandLine = "" 
                        writeFile = open("{}ServiceMgrContainer_{}.log".format(LogsPath, VMDevice['IP']), "w")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()
                        
                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}CliContainers_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "add", ServiceMgrDesc, ProcessTail)
                        writeFile = open(batCliContainersCommand, "a")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()

                        if Setup['TestMgrOS'] == "Linux":
                            ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            ls_output.communicate()
                        elif Setup['TestMgrOS'] == "Win":
                            ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            ls_output.communicate()
                        
                        cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "start", ServiceMgrDesc, ProcessTail)
                        writeFile = open(batCliContainersCommand, "w")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()

                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}ServiceMgrContainer_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliContainersCommandLine="{} {} plink -t -batch -pw {} {}@{} {}WaitServiceMgrContainer{} {} {}\n".format(startCommand, ProcessTitle, VMDevice['Password'], VMDevice['Username'], VMDevice['IP'], VMHomeFilesPath, VMFileExt, ServiceMgrDesc, ProcessTail)
                        writeFile = open(batCliContainersCommand, "a")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()
                        
                        if Setup['TestMgrOS'] == "Linux":
                            ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        elif Setup['TestMgrOS'] == "Win":
                            ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        time.sleep(5)
                        
                        DeviceJoin = 0
                        while DeviceJoin == 0:
                            with open("{}ServiceMgrContainer_{}.log".format(LogsPath, VMDevice['IP'])) as f:
                                content = f.read()
                                if 'ServiceMgr is Running... Please don\'t close it' in content:
                                    DeviceJoin = 1
                                else:
                                    print("Waiting for the ServiceMgr Container on {}".format(VMDevice['IP']))
                                    time.sleep(3)
                                    
                        print("The ServiceMgr Container on {} is running successfully".format(VM['IP']))

Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
Waiting for the ServiceMgr Container on 147.172.178.148
The ServiceMgr Container on 147.172.178.145 is running successfully


# Add Services

In [ ]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['RunExamples']:
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    
                    for Ex in Examples:
                        if not Ex['Included']:
                            continue
                        if not Ex['Run']:
                            continue
                        ServiceURL = "http://{}:4200/download/{}".format(Setup['MgtUiServerIP'], Ex['Artifact'])

                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}AddService_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "add", ServiceURL, ProcessTail)
                        writeFile = open(batCliServicesCommand, "a")
                        writeFile.write(cliServicesCommandLine)
                        writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            
            for Ex in Examples:
                if not Ex['Included']:
                    continue
                if not Ex['Run']:
                    continue
                    
                print("{} example application is added to the platform".format(Ex['Name']))

# StartAll Services

In [ ]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['RunExamples']:
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    
                    for Ex in Examples:
                        if not Ex['Included']:
                            continue
                        if not Ex['Run']:
                            continue
                        ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}StartAllService_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "startAll", ArtifactDeployment, ProcessTail)
                        writeFile = open(batCliServicesCommand, "a")
                        writeFile.write(cliServicesCommandLine)
                        writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
                

# StopAll Services

In [ ]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['RunExamples']:
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    
                    for Ex in Examples:
                        if not Ex['Included']:
                            continue
                        if not Ex['Run']:
                            continue
                        ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}StopAllService_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "stopAll", ArtifactDeployment, ProcessTail)
                        writeFile = open(batCliServicesCommand, "a")
                        writeFile.write(cliServicesCommandLine)
                        writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
                

# Remove Services

In [ ]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['RunExamples']:
    cliServicesCommandLine = "" 
    writeFile = open(batCliServicesCommand, "w")
    writeFile.write(cliServicesCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    
                    for Ex in Examples:
                        if not Ex['Included']:
                            continue
                        if not Ex['Run']:
                            continue
                        ArtifactDeployment = "{}/{}".format(Setup['ArtifactsFolder'], Ex['Artifact'])

                        if Setup['TestMgrOS'] == "Linux":
                            ProcessTail = " > {}RemoveService_{}.log &".format(LogsPath, VMDevice['IP'])
                        cliServicesCommandLine="plink -t -batch -pw {} {}@{} {}CliServicesCommand{} {} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "remove", ArtifactDeployment, ProcessTail)
                        writeFile = open(batCliServicesCommand, "a")
                        writeFile.write(cliServicesCommandLine)
                        writeFile.close()

            if Setup['TestMgrOS'] == "Linux":
                ls_output=subprocess.Popen(['sh', batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
            elif Setup['TestMgrOS'] == "Win":
                ls_output=subprocess.Popen([batCliServicesCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                ls_output.communicate()
                

# Stop Service Manager Container

In [19]:
# Run script to start Services Manager for Device(s) with contianer version
if Setup['StopServiceMgrContainer']:
    cliContainersCommandLine = "" #plink -t -batch -pw {} {}@{} \"chmod 755 {}Files/*{}\"\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomePath, VMFileExt)
    writeFile = open(batCliContainersCommand, "w")
    writeFile.write(cliContainersCommandLine)
    writeFile.close()
    
    for VM in Machines:
        if not VM['Included']:
            continue
        
        ServiceMgrDesc = "http://{}:4200/download/SimpleMeshTestingApp_dflt_mesh-info.yml".format(Setup['MgtUiServerIP'])
        if VM['Role'] == "Server":
            for VMDevice in Machines:
                if not VMDevice['Included']:
                    continue
                    
                if VMDevice['Role'] == "Device":
                    if VMDevice['RunAsContainer']:
                        cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "stop", ServiceMgrDesc)
                        writeFile = open(batCliContainersCommand, "a")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()
                        # Create bat file to stop Ecs container on the Device(s)
                        cliContainersCommandLine="plink -t -batch -pw {} {}@{} {}CliContainersCommand{} {} {} {}\n".format(VM['Password'], VM['Username'], VM['IP'], VMHomeFilesPath, VMFileExt, VMDevice['ResourceID'], "undeploy", ServiceMgrDesc)
                        writeFile = open(batCliContainersCommand, "a")
                        writeFile.write(cliContainersCommandLine)
                        writeFile.close()

                        if Setup['TestMgrOS'] == "Linux":
                            ls_output=subprocess.Popen(['sh', batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            ls_output.communicate()
                        elif Setup['TestMgrOS'] == "Win":
                            ls_output=subprocess.Popen([batCliContainersCommand], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                            ls_output.communicate()
                

# Resources Test

In [20]:
'''
if Setup['RunTest']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batTestResources], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batTestResources], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
        
    found = 0
    included = 0
    
    for VM in Machines:
        if VM['Role'] == "Device":
            if not VM['Included']:
                continue
            included += 1
            with open(testResultsFile) as f:
                if VM['IP'] in f.read():
                    found += 1
    
    print("{} device(s) out of {} device(s) is/are joined the platform successfully".format(found, included))
'''
print("Next")

Next


# Device(s) Leave The Platform

In [21]:
# Run script to stop and leave the Device(s) from the platform
if Setup['DevicesLeavePlatform']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batLeavePlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batLeavePlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()


# Stop The Platform

In [22]:
# Run script to stop the platform Server
if Setup['StopPlatfrom']:
    if Setup['TestMgrOS'] == "Linux":
        ls_output=subprocess.Popen(['sh', batStopPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()
    elif Setup['TestMgrOS'] == "Win":
        ls_output=subprocess.Popen([batStopPlatform], shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        ls_output.communicate()